To do
* go through deleted posts to verify missing values
* Create indicator variables

Importing libraries

In [3]:
import numpy as np
import pandas as pd

# Webscraping

## Installing libraries

In [ ]:
pip install praw

In [ ]:
import praw

In [ ]:
cred = np.genfromtxt('cred.txt',dtype=str,delimiter=',')

Creating a Reddit Instance (need to hide the details in the cell later at some point)

In [ ]:
reddit = praw.Reddit(client_id=cred[0], client_secret=cred[1], user_agent=cred[2])

Let's play around with this

In [ ]:
hot_posts = reddit.subreddit('StableDiffusion').hot(limit=100)
for post in hot_posts:
  print(post.id)

In [ ]:
submission = reddit.submission(id='xcq819')
for y in submission.comments:
  print(y.body)

In [ ]:
type(submission.comments[0].body)

Hmmm after observing the "Prompt Included" posts, it seems that users are putting the prompts in the following places:


*   title
*   in a body of text where the images are linked elsewhere
*   comments

We're going to need to account for all of this later down the road



In [ ]:
posts = []
sd_subreddit = reddit.subreddit('StableDiffusion')
for post in sd_subreddit.hot(limit=1000):
  if post.link_flair_text == 'Prompt Included':
    posts.append([post.title,post.comments,post.url,post.id,post.score,post.num_comments,post.over_18])
    
posts = pd.DataFrame(posts,columns=['title',
                                    'comments',
                                    'url',
                                    'id',
                                    'score',
                                    'num_comments',
                                    'nsfw'])

In [ ]:
posts.head()

,title,comments,url,id,score,num_comments,nsfw
0,'This looks like a nice place for a nap.',"(iqzlmuh, ir0l1pa, ir0sc0q)",https://i.redd.it/if84ow9nnqr91.png,xv86kw,85,7,False
1,Just some cute hand sewn pumpkins,"(ir0x84g, ir190zu)",https://www.reddit.com/gallery/xvhq8v,xvhq8v,11,2,False
2,How to put HEAVY BlingBling golden chains arou...,"(iqz85k2, iqzhgiq, iqzs27l, iqzwn3g)",https://www.reddit.com/gallery/xv5bbd,xv5bbd,63,10,False
3,Dragonball Sesamestreet mash-up,(ir0euko),https://i.redd.it/v1hpyux5rqr91.jpg,xv8hcz,33,2,False
4,Christoph Waltz as a high ranking Imperial off...,(),https://i.redd.it/iha29x8vgtr91.jpg,xvjrpd,6,0,False


In [ ]:
posts.shape

(162, 7)

In [ ]:
comments = []
for i in posts.id:
  submission = reddit.submission(id=i)
  for j in submission.comments:
    comments.append([j.body,i])
comments = pd.DataFrame(comments,columns=['text','id'])  


In [ ]:
comments.iloc[0,0]

'Created with Stable Diffusion using the NMKD Stable Diffusion GUI - AI Image Generator.\r  \n\r  \nPrompt:\r  \nfemale ent, insanely detailed and intricate, hypermaximalist, elegant, ornate, hyper realistic, super detailed,, 8K, vivid colors, sunrise, open sky, black bear\r  \n\\-1029314660 -scale 7.50 -k euler a  \n\n\nYes, the female ent is supposed to be in there somewhere, but I never found her either.'

In [ ]:
comments.to_csv('/content/comments.csv')

It looks like comments are being cut off once converted to a csv

In [ ]:
comments.to_excel('/content/comments.xlsx')

Okay it looks like the whole comment is intact if you leave it in an Excel file format.

Okay so it looks like to get past the 1000 limit we can use PSAW (Pushshift) and get an index of id and then use PRAW to look at those posts and comments more in-depth. So let's figure out how the API works:

In [ ]:
pip install psaw

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Creating the raw dataset

In [ ]:
from psaw import PushshiftAPI

In [ ]:
api = PushshiftAPI(reddit)

In [ ]:
#Grab a list of the ids of all posts from the Stable Diffusion subreddit
submissionList = list(api.search_submissions(subreddit='StableDiffusion'))

# Create temporary list
possiblePromptPost = []

# Get posts with "Prompt Included" flair
for post in submissionList:
  if post.link_flair_text == 'Prompt Included':
    possiblePromptPost.append(post)

In [ ]:
temp = []
# Iterate through each post from the list of post ids obtained with PSAW
for submission in possiblePromptPost:

  # Using PRAW to create a submission object to get the text content
  post = reddit.submission(id = submission)

  # Add the text as a list
  temp.append([post.title,post.id,post.selftext,post.comments])
df = pd.DataFrame(temp, columns = ['title','id','text','comments'])
df

In [ ]:
# Export the dataframe to an excel file
df.to_excel('/content/prompt_data.xlsx')

## Adding text to the "comments" column

Up to this point, the "comments" column only has ids of the comments so we're going to have to iterate through them and add them.

In [ ]:
df = pd.read_excel('/content/prompt_data.xlsx')

In [ ]:
df.head()

,Unnamed: 0,title,id,text,comments
0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,<praw.models.comment_forest.CommentForest obje...
1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,<praw.models.comment_forest.CommentForest obje...
2,2,Lovecraftian Behemoth,xvqu7g,NaN,<praw.models.comment_forest.CommentForest obje...
3,3,Henry Cavill medieval times(?,xvqpvg,NaN,<praw.models.comment_forest.CommentForest obje...
4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,<praw.models.comment_forest.CommentForest obje...


In [ ]:
print(df.comments[0])

We want the OP's comment since that's usually where the prompt is if they have decided to include the prompt in the comments.

In [ ]:
#Getting the OP's comments from the posts if any

#Create temporary list
temp = []

#Iterate through the first 50 posts
for id in df.id:

  #Create a submission object with the id
  submission = reddit.submission(id = id)

  #Check if the comment tree is empty
  if (len(submission.comments) == 0):
    temp.append("NA")
  else:
    # Create a temp list to store OP's comments' epoch times
    utc = []

    # Create a temp list to store OP's comments' ids
    op_comment_ids = []

    # Iterate through all the comments
    for comment in submission.comments:

      # Check if the comment is the OP
      if comment.is_submitter:

        # Add the epoch time to the list
        utc.append(comment.created_utc)

        # Add the comment's id to the op comment id list
        op_comment_ids.append(comment.id)

    for op_comment_id in op_comment_ids:
      curr_comment = reddit.comment(op_comment_id)
      # Check if the comment
      if curr_comment.created_utc == min(utc):

        # Append the comment to the temp list
        temp.append(curr_comment.body)
    
    # Append "NA" if there are no OP comments 
    if len(utc) == 0:
      temp.append("NA")
  
df.loc[:,'comments'] = temp
df.to_excel('/content/new_prompt_data.xlsx')


References:

[Reference used for webscraping Reddit](https://towardsdatascience.com/scraping-reddit-data-1c0af3040768)

[PRAW Documentation](https://praw.readthedocs.io/en/latest/index.html)

[Submission attributes](https://praw.readthedocs.io/en/latest/code_overview/models/submission.html)

[Getting posts with a specific flair](https://www.reddit.com/r/redditdev/comments/x4vydg/how_do_i_pull_posts_with_a_specific_flair_from_a/)

[Access more than 100 posts](https://www.reddit.com/r/redditdev/comments/t3j440/how_to_access_more_than_100_posts_with_praw/)

[How to Scrape Large Amounts of Reddit Data](https://medium.com/swlh/how-to-scrape-large-amounts-of-reddit-data-using-pushshift-1d33bde9286)

[Getting many/all submissions from a subreddit using PRAW/PSAW/pushshift](https://www.reddit.com/r/redditdev/comments/knbns9/getting_manyall_submissions_from_a_subreddit/)

[Using Pushshift API for data analysis on Reddit](https://medium.com/mcd-unison/using-pushshift-api-for-data-analysis-on-reddit-b08d339c48b8)

[Video: Reddit Scraping using PRAW and Pushshift](https://www.youtube.com/watch?v=a3zQdlLwEb4)

[Pushshift Reddit API v4.0 Documentation](https://reddit-api.readthedocs.io/en/latest/#search-parameters-for-comments)

[New to Pushshift? Read this! FAQ](https://www.reddit.com/r/pushshift/comments/bcxguf/new_to_pushshift_read_this_faq/)




# Data Cleaning

In [ ]:
df = pd.read_excel('new_prompt_data.xlsx')

In [ ]:
pd.set_option('display.max_rows', 500)

In [ ]:
df[df.text == '[deleted]']

,Unnamed: 0,Unnamed: 0.1,title,id,text,comments
19,19,19,Steampunk Waifu,xvak6q,[deleted],NaN
22,22,22,Fish & Dahlia,xv8ocu,[deleted],NaN
35,35,35,The Seething Human Artist,xv26u6,[deleted],NaN
39,39,39,"[NSFL] Crying baby stuck in a washing machine,...",xv05wm,[deleted],NaN
54,54,54,"Desert scene, text2img --> img2img",xurc0a,[deleted],NaN
63,63,63,"Something diferente, but make with Waifu Diffu...",xul4iy,[deleted],NaN
89,89,89,A majestic Elvish city with waterfalls in a fo...,xtz0o5,[deleted],NaN
141,141,141,"Quote Prompt: ""Whoever fights monsters should ...",xszmfx,[deleted],NaN
155,155,155,Scary Robot,xsp4jw,[deleted],NaN
156,156,156,scary robot,xsp409,[deleted],NaN


In [ ]:
df[df.text == '[deleted]'].shape

(271, 6)

It looks like for some posts there are prompts in the comments but because it reads the OP as "u/deleted" it saves the comment text as NaN in our algorithm block from the webscraping section. Looks like we're going to have to fix this

In [ ]:
df[df.text == '[deleted]'].to_excel('/content/deleted_posts.xlsx')

In [182]:
pd.set_option('display.max_colwidth', 50)

In [183]:
deleted_posts = pd.read_excel('/content/deleted_posts.xlsx')

In [184]:
deleted_posts.head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,title,id,text,comments
0,19,19,19,Steampunk Waifu,xvak6q,[deleted],NaN
1,22,22,22,Fish & Dahlia,xv8ocu,[deleted],NaN
2,35,35,35,The Seething Human Artist,xv26u6,[deleted],has prompt
3,39,39,39,"[NSFL] Crying baby stuck in a washing machine,...",xv05wm,[deleted],NaN
4,54,54,54,"Desert scene, text2img --> img2img",xurc0a,[deleted],NaN


In [185]:
deleted_posts_indices = deleted_posts[deleted_posts.comments.isna()].set_index('Unnamed: 0').index

In [186]:
df = pd.read_excel('/content/new_prompt_data.xlsx')

In [187]:
df.head()

,Unnamed: 0,Unnamed: 0.1,title,id,text,comments
0,0,0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,1,1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,2,2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,3,3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,4,4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."


In [188]:
df = df.drop(columns = ['Unnamed: 0','Unnamed: 0.1'])

In [189]:
df = df.drop(deleted_posts_indices).reset_index().drop(columns=['index'])

In [190]:
df

,title,id,text,comments
0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."
...,...,...,...,...
2171,Renaissance art of Keanu Reeves,woy7hy,NaN,"prompt: !dream portrait of keanu reeves, oil p..."
2172,Jazz robots.,wo3wm3,NaN,Prompt: !dream 3 steampunk robot jazz musician...
2173,Raves celebrating the day of the dead,wnei33,NaN,"prompt I used: !dream scene from a rave, celeb..."
2174,Joker artwork,wm97qi,NaN,This is the prompt I used !dream symmetry!! st...


There was one row that I saw that I'd also like to remove:

In [191]:
df = df.drop(df[df.id == 'x9e2c2'].index).reset_index().drop(columns=['index'])

I removed some of the duplicate posts in removing deleted posts but I want to make sure there aren't any still remaining:

In [192]:
df.shape

(2175, 4)

In [193]:
df.title.nunique()

2166

## Posts with duplicate titles

In [194]:
df[df.title.duplicated()]

,title,id,text,comments
276,this is not art,xqsbvi,NaN,NaN
307,Beauty Girl #drawing #sketch #sketchbook #port...,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde ha..."
404,"a shiba inu taking a bath in a tub, top view, ...",xo85rx,NaN,NaN
455,Going Dali,xngvq1,[deleted],"​prompt ""surreal salvador dali painting of a f..."
1484,Stripperella,x81vif,NaN,**PROMPT**: Stripperella by Fabian Perez \n**...
1682,Manual Upscaled background using Stable Diffusion,x63vpk,I had a background that I liked although in l...,NaN
1747,:v,x5ll9e,NaN,"a beautiful woman, well-fitting white dress, s..."
1902,An oil painting of A teen girl sitting and gaz...,x3t3ee,[deleted],NaN
2081,Female elven warriors,x1rjgs,NaN,NaN


In [195]:
list(df[df.title.duplicated()].index)

[276, 307, 404, 455, 1484, 1682, 1747, 1902, 2081]

In [196]:
df[df.title == 'this is not art']

,title,id,text,comments
202,this is not art,xrqxvy,NaN,this is not art \nNegative prompt: art \nSte...
276,this is not art,xqsbvi,NaN,NaN


In [197]:
df[df.title == df.iloc[307].title]

,title,id,text,comments
258,Beauty Girl #drawing #sketch #sketchbook #port...,xr0dwo,NaN,"`prompts : ""a painting of a woman with pink ha..."
307,Beauty Girl #drawing #sketch #sketchbook #port...,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde ha..."


In [198]:
df[df.title == df.iloc[404].title].shape[0]

2

In [199]:
pd.set_option('display.max_colwidth', None)

In [200]:
temp = pd.DataFrame()
for index in list(df[df.title.duplicated()].index):
  for x in range(df[df.title == df.iloc[index].title].shape[0]):
    temp = temp.append(df[df.title == df.iloc[index].title].iloc[x])
temp

,title,id,text,comments
202,this is not art,xrqxvy,NaN,"this is not art \nNegative prompt: art \nSteps: 20, Sampler: Euler a, CFG scale: 7, Seed: 3121606117, Size: 512x512"
276,this is not art,xqsbvi,NaN,NaN
258,Beauty Girl #drawing #sketch #sketchbook #portrait #beautiful #beauty #girl #art #concept #digitalart #digital #artwork #artist #stablediffusion #artoftheday #laureartwork #artstyles #artstyle #photoshoppainting #digitalillustrations,xr0dwo,NaN,"`prompts : ""a painting of a woman with pink hair, a digital painting , featured on cgsociety, fantasy art, white background, sensual""`"
307,Beauty Girl #drawing #sketch #sketchbook #portrait #beautiful #beauty #girl #art #concept #digitalart #digital #artwork #artist #stablediffusion #artoftheday #laureartwork #artstyles #artstyle #photoshoppainting #digitalillustrations,xq5aw4,NaN,"`Prompts ""a painting of a woman with blonde hair, a digital painting , featured on cgsociety, fantasy art, speedpainting, digital painting""`"
390,"a shiba inu taking a bath in a tub, top view, professional photography, 4k",xoml67,NaN,NaN
404,"a shiba inu taking a bath in a tub, top view, professional photography, 4k",xo85rx,NaN,NaN
454,Going Dali,xnj9if,NaN,"​python optimizedSD/optimized\_txt2img.py --​​prompt ""surreal salvador dali painting of a female figure sitting with a scarf draped over her shoulder and in the background are dark clouds and a plain brown surface and many clocks and skulls melting"" --H 768 --W 1280 --seed 27 --n\_iter 2 --n\_samples 1 --ddim\_steps 50"
455,Going Dali,xngvq1,[deleted],"​prompt ""surreal salvador dali painting of a female figure sitting with a scarf draped over her shoulder and in the background are dark clouds and a plain brown surface and many clocks and skulls melting"""
1483,Stripperella,x81wd2,NaN,"**PROMPT**: Stripperella by Fabian Perez, seductive lingerie, portrait, luminous necklace, luminous butterflies, film grain, closup, focus blur \n**SAMPLER**: PLMS \n**WIDTH**: 512 \n**HEIGHT**: 512 \n**STEPS**: 100 \n**SCALE**: 12 \n**SEED**: 2608420685"
1484,Stripperella,x81vif,NaN,**PROMPT**: Stripperella by Fabian Perez \n**SAMPLER**: PLMS \n**WIDTH**: 512 \n**HEIGHT**: 512 \n**STEPS**: 100 \n**SCALE**: 12 \n**SEED**: 1070490662


In [201]:
temp.shape

(18, 4)

In [203]:
pd.set_option('display.max_colwidth', 50)

In [205]:
df = df.drop([276,404,455,1484,1682,1902,2081]).reset_index().drop(columns=['index'])

## Posts with duplicate comments

In [206]:
df.head()

,title,id,text,comments
0,Man needlessly standing on roof overlooking cy...,xvrn98,NaN,Generated with:\n\n> Portrait of a man overloo...
1,"first attempt at art nouveau prompts, in comments",xvrbxa,NaN,NaN
2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."


In [207]:
df.shape

(2168, 4)

In [210]:
df[df.comments.notna().duplicated()]

,title,id,text,comments
2,Lovecraftian Behemoth,xvqu7g,NaN,Ultra realistic Illustration of a Lovecraftian...
3,Henry Cavill medieval times(?,xvqpvg,NaN,"wasn't expecting this results when I put ""Henr..."
4,Android Pippi Longstocking in Neo Tokyo,xvpdis,NaN,"Prompt: android pippi longstocking samurai, ne..."
5,Neo gothic buildings,xvojq5,NaN,"neo-Gothic building, industrial area, rain, mo..."
6,D&D character animation with Stable Diffusion,xvo91q,NaN,NaN
...,...,...,...,...
2163,Renaissance art of Keanu Reeves,woy7hy,NaN,"prompt: !dream portrait of keanu reeves, oil p..."
2164,Jazz robots.,wo3wm3,NaN,Prompt: !dream 3 steampunk robot jazz musician...
2165,Raves celebrating the day of the dead,wnei33,NaN,"prompt I used: !dream scene from a rave, celeb..."
2166,Joker artwork,wm97qi,NaN,This is the prompt I used !dream symmetry!! st...


In [229]:
pd.set_option('display.max_colwidth', None)
temp_df = df[df.comments.notna()]
temp_df[temp_df.comments.duplicated()]

,title,id,text,comments
59,"Something diferente, but make with Waifu Diffusion",xul4iy,[deleted],"Prompt = an android ninja in Rome at night. symmetry. realistic proportions. digital art , cgsociety, fantasy art, daz3d, behance hd, photoillustration"
86,a giant face made of clouds painting by beksiński,xtxab0,NaN,"Prompt: a giant face made of clouds, realistic painting by beksiński, highly detailed, 4k, realistic sad eyes, crystal eyes, front view"
165,SD Built Me a Sci Fi World and Flew Me Round it in Less than an Hour (watch out CIG),xsgk1k,NaN,"prompt: very close photo of a small detailed spaceship flying over a brutalist cyberpunk city, view of brutalist skyline, fluorescent accents, billboards, dramatic sky, cinematic lighting, volumetric lighting, global illumination, depth of field, sharp focus, ray tracing"
392,“The Imminent Tribe” (Upscaled),xohh1r,NaN,"Prompt = a beautiful street view of a cyberpunk Japanese street market at night, open wide sky, realism, volumetric lighting, moody, cinematic, composition, intricate, insane level of detail, artstation HQ, 16K, depth of field"
419,"Deconstructing Hypercubic Ferrofluids Part 2/3: Ferroflowers, Ferroparrots and Ferro-Hello-Kitty.",xnyeqm,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
467,Deconstructing Hypercubic Ferrofluids,xn8por,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
498,Ana De Armas' photoshoot in Night City pt 01,xmqji5,NaN,"prompt: stunning ethereal photo of Ana De Armas wearing a detailed sci fi outfit, jacket, hotpants, in a cyberpunk setting, shop fronts with fluorescent signs, vibrant, cinematic lighting, global illumination, rim light, ray tracing, depth of field, sharp focus, bokeh, sub surface scattering, neon accents"
946,'The Art of Waves',xe2pnq,NaN,generated with: [https://apps.apple.com/us/app/ai-dreamer/id1608856807](https://apps.apple.com/us/app/ai-dreamer/id1608856807)
1076,Gods of Chaos,xc9lc4,NaN,More stuff + prompts on IG https://www.instagram.com/voidrov/
1205,“Surviving the Ultimatum”,xareok,NaN,"Prompt = a beautiful humongous blue whale jumping out of water, by Ivan Aivazovsky, hypermaximalist, a sense of scale and awe, hyper realistic, intricate, insane level of details, artstation HQ"


In [233]:
temp = pd.DataFrame()
for index in list(temp_df[temp_df.comments.duplicated()].index):
  for x in range(df[df.comments == df.iloc[index].comments].shape[0]):
    temp = temp.append(df[df.comments == df.iloc[index].comments].iloc[x])
temp

,title,id,text,comments
57,It's something diferent,xulhuw,NaN,"Prompt = an android ninja in Rome at night. symmetry. realistic proportions. digital art , cgsociety, fantasy art, daz3d, behance hd, photoillustration"
59,"Something diferente, but make with Waifu Diffusion",xul4iy,[deleted],"Prompt = an android ninja in Rome at night. symmetry. realistic proportions. digital art , cgsociety, fantasy art, daz3d, behance hd, photoillustration"
28,"a giant face made of clouds, realistic painting by beksiński",xv69ba,NaN,"Prompt: a giant face made of clouds, realistic painting by beksiński, highly detailed, 4k, realistic sad eyes, crystal eyes, front view"
86,a giant face made of clouds painting by beksiński,xtxab0,NaN,"Prompt: a giant face made of clouds, realistic painting by beksiński, highly detailed, 4k, realistic sad eyes, crystal eyes, front view"
161,"And with a few Typed Words, on we go to Build Another Sci Fi World",xsjhir,NaN,"prompt: very close photo of a small detailed spaceship flying over a brutalist cyberpunk city, view of brutalist skyline, fluorescent accents, billboards, dramatic sky, cinematic lighting, volumetric lighting, global illumination, depth of field, sharp focus, ray tracing"
165,SD Built Me a Sci Fi World and Flew Me Round it in Less than an Hour (watch out CIG),xsgk1k,NaN,"prompt: very close photo of a small detailed spaceship flying over a brutalist cyberpunk city, view of brutalist skyline, fluorescent accents, billboards, dramatic sky, cinematic lighting, volumetric lighting, global illumination, depth of field, sharp focus, ray tracing"
349,“The Imminent Tribe 2” (Upscaled),xpeole,NaN,"Prompt = a beautiful street view of a cyberpunk Japanese street market at night, open wide sky, realism, volumetric lighting, moody, cinematic, composition, intricate, insane level of detail, artstation HQ, 16K, depth of field"
392,“The Imminent Tribe” (Upscaled),xohh1r,NaN,"Prompt = a beautiful street view of a cyberpunk Japanese street market at night, open wide sky, realism, volumetric lighting, moody, cinematic, composition, intricate, insane level of detail, artstation HQ, 16K, depth of field"
381,"Deconstructing Hypercubic Ferrofluids Part 3/3: FerroAmerica, Ferropoliticians, and FerroPutin with Ferrobullets (among others)",xov4k5,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
419,"Deconstructing Hypercubic Ferrofluids Part 2/3: Ferroflowers, Ferroparrots and Ferro-Hello-Kitty.",xnyeqm,NaN,"Prompt: *""<Subject>. Hypercube ferrofluid. Motion blur. By Alberto Seveso. Bokeh. Depth of focus. Sharp focus. Tilt shift. Macrophotography. 4K. 8K. HDR. Postprocessed. Volumetric lighting""*"
